# Unsupervised Learning notebook

This notebook will be for finding inferences on the given obesity dataset with unsupervised learning algorithms K-Means and Hierarchical clustering

#### Initial import and data check

In [1]:
# initial imports
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, homogeneity_score, completeness_score

import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
# import obesity data
obesity_df = pd.read_csv("assignment_dataset/ObesityDataSet_raw_and_data_sinthetic.csv")

In [3]:
obesity_df.shape

(2111, 17)

In [4]:
obesity_df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


#### Set feature types

We need to define what types of variables are features are so we can transform them correctly for the distance algorithms

In [5]:
# Define feature groups based on data types
numerical_features = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
binary_features = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
ordinal_features = ['CAEC', 'CALC']
nominal_features = ['Gender', 'MTRANS']

In [6]:
# Check for 16 features
len(numerical_features) + len(binary_features) + len(ordinal_features) + len(nominal_features)

16

#### Encodings

In [7]:
# work on copy
encoded_df = obesity_df.copy()

##### Binary Features

In [8]:
binary_map = {'yes':1, 'no':0}

for col in binary_features:
    encoded_df[col] = encoded_df[col].map(binary_map).astype(int)

In [12]:
encoded_df[binary_features]

,family_history_with_overweight,FAVC,SMOKE,SCC
0,1,0,0,0
1,1,0,1,1
2,1,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
2106,1,1,0,0
2107,1,1,0,0
2108,1,1,0,0
2109,1,1,0,0


##### Ordinal features

For the two ordinal features

no < sometimes < frequently < always -> 0, 1, 2, 3

In [10]:
ordinal_map = {
    'no':0,
    'Sometimes':1,
    'Frequently':2,
    'Always':3
}

for col in ordinal_features:
    encoded_df[col] = encoded_df[col].map(ordinal_map)

In [13]:
encoded_df[ordinal_features]

,CAEC,CALC
0,1,0
1,1,1
2,1,2
3,1,2
4,1,1
...,...,...
2106,1,1
2107,1,1
2108,1,1
2109,1,1


##### Nominal features - One hot encoding

Use one hot encoding for the nominal features and drop first. Absense of any 1 value implies the dropped value

In [14]:
encoded_df = pd.get_dummies(encoded_df, columns=nominal_features, drop_first=True)

In [16]:
encoded_df.columns.to_list()

['Age',
 'Height',
 'Weight',
 'family_history_with_overweight',
 'FAVC',
 'FCVC',
 'NCP',
 'CAEC',
 'SMOKE',
 'CH2O',
 'SCC',
 'FAF',
 'TUE',
 'CALC',
 'NObeyesdad',
 'Gender_Male',
 'MTRANS_Bike',
 'MTRANS_Motorbike',
 'MTRANS_Public_Transportation',
 'MTRANS_Walking']

#### Create feature dataset - a.k.a X

In [17]:
# Get all feature columns except NObeyesdad
feature_cols = [col for col in encoded_df.columns if col != 'NObeyesdad']

len(feature_cols) # should be 19

19

In [18]:
X = encoded_df[feature_cols]

In [20]:
X.shape # should be 2111 rows by 19 columns

(2111, 19)

X### K-Means